## Cifar10이미지를 가지고 keras로 학습하고 예측해본다. 

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

import os

Using TensorFlow backend.
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\fra

- 코랩을 이용할 거면 구글드라이버 마운트해서 해야함

```
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')
```
```
cd gdrive/My Drive/Colab Notebooks/models
```

In [2]:
batch_size = 16
num_classes = 10
epochs = 25

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
print(x_train.shape)
print(x_train.shape[0])
print(x_test.shape)
print(x_test.shape[0])
print(y_train.shape)
print(y_train.shape[0])
print(y_test.shape)
print(y_test.shape[0])
x_train.shape[1:]

(50000, 32, 32, 3)
50000
(10000, 32, 32, 3)
10000
(50000, 1)
50000
(10000, 1)
10000


(32, 32, 3)

In [5]:
# categorical_entoropy를 써야하기에 to_categorical 해줘야 함.
# sparse_categorical_crossentopy를 쓸 때는 안해줘도 됨. 알아서 해줌
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train[:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [26]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [27]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

Using real-time data augmentation.


In [28]:
filepath = "./saved_model/cifar_saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc',
                                            verbose=1, save_best_only=False,
                                            mode='max')

In [29]:
# 네트워크를 직접 설계해보자.
n_kernels = 32
model = Sequential()

# input : 32, 32, 3
model.add(Conv2D(n_kernels, (3,3), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=(x_train.shape[1:])))
model.add(Conv2D(n_kernels*2, (3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# input : 32, 32, 64
model.add(Conv2D(n_kernels*2, (3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(Conv2D(n_kernels*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# input : 8, 8, 128
model.add(Conv2D(n_kernels*8, (3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# [중요] GlobalAveragePooling2D() Flatten과 달리 앞 단계 output을 펼치는 것이 아니라
# 앞 단계 Conv2D 도출될 피쳐맵(여기선 3*3*256)에서 각 피쳐(3*3)의 평균으로 256노드에 순차적으로 넣어줌)
# 그런데 lobalAveragePooling2D() 하면 256노드
# Flatten하면 3*3*256 노드

model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation='softmax', kernel_initializer='glorot_uniform'))

In [30]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_10 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
batch_normalization_11 (Batc (None, 12, 12, 128)      

In [31]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [32]:
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['acc'])

In [33]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                   epochs=epochs, validation_data=(x_test, y_test),
                   workers=4, callbacks=[checkpoint])

Epoch 1/25
3125/3125 [==============================] - 29s 9ms/step - loss: 1.5580 - acc: 0.4413 - val_loss: 2.9118 - val_acc: 0.2979

Epoch 00001: saving model to ./saved_model/cifar_saved-model-01-0.30.hdf5
Epoch 2/25
3125/3125 [==============================] - 28s 9ms/step - loss: 1.2899 - acc: 0.5432 - val_loss: 2.8104 - val_acc: 0.3502

Epoch 00002: saving model to ./saved_model/cifar_saved-model-02-0.35.hdf5
Epoch 3/25
3125/3125 [==============================] - 28s 9ms/step - loss: 1.1582 - acc: 0.5957 - val_loss: 1.4073 - val_acc: 0.5240

Epoch 00003: saving model to ./saved_model/cifar_saved-model-03-0.52.hdf5
Epoch 4/25
3125/3125 [==============================] - 28s 9ms/step - loss: 1.0807 - acc: 0.6217 - val_loss: 2.6609 - val_acc: 0.3905

Epoch 00004: saving model to ./saved_model/cifar_saved-model-04-0.39.hdf5
Epoch 5/25
3125/3125 [==============================] - 28s 9ms/step - loss: 1.0186 - acc: 0.6439 - val_loss: 3.1851 - val_acc: 0.3264

Epoch 00005: saving mode